## Learning Rate Sensitivity of RPS-LJE
Table 6 (appendix)

In [ ]:
import numpy as np
import torch
from models.RNN.utils_imdb import load_data



In [3]:
train_data, valid_data, test_data =load_data()

In [4]:
path = '../../saved_models/base'
file = np.load('{}/model/saved_outputs.npz'.format(path))
intermediate_train = torch.from_numpy(file['intermediate_train']).squeeze()
intermediate_test = torch.from_numpy(file['intermediate_test']).squeeze()
labels_train = file['labels_train']
labels_test = file['labels_test']
pred_train = file['pred_train'].squeeze()
pred_test = file['pred_test'].squeeze()

In [24]:
def get_ours_order(weight_matrix_ours, test_point=None):
    true_class = labels_test[test_point]
    tmp = np.dot(weight_matrix_ours,
                 intermediate_test[test_point,:])
    if true_class == 1:
        pos_idx = np.flip(np.argsort(tmp), axis=0)
    else:
        pos_idx = np.argsort(tmp)
    return pos_idx

In [25]:
def get_text(data):
    return ' '.join([str(elem) for elem in data.text])[:200]
def get_label(data):
    sentiment = {'neg':0,'pos':1}
    return sentiment[data.label]
    # return data.label

In [26]:
wrongly_predicted_train_ids = np.argwhere(np.abs(np.round(pred_train)-labels_train)>0).flatten()
wrongly_predicted_test_ids = np.argwhere(np.abs(np.round(pred_test)-labels_test)>0).flatten()

In [27]:
lr_list = [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
weight_matrix_ours_list = [np.load('{}/calculated_weights/ours_weight_matrix_with_lr_{}.npz'.
                                   format(path, lr), allow_pickle=True)['weight_matrix'].
                               squeeze() for lr in lr_list]

In [28]:
import pandas as pd
test_pt = 5

columns=['lr','type','id','sentiment class','review text']
data_array_ours = []
data_array_rep = []
data_array_IF = []

data_array_ours.append([' ','Test point',test_pt,
                        get_label(test_data[test_pt]),
                        get_text(test_data[test_pt])[:200]])


for lr_idx in range(len(lr_list)):
    ours_idx_pos = [i for i in get_ours_order( weight_matrix_ours_list[lr_idx], test_pt) if not i in wrongly_predicted_train_ids]
    for i in range(3):
        our_data = train_data[ours_idx_pos[i]]
        data_array_ours.append([lr_list[lr_idx],'Positive {}'.format(i+1), ours_idx_pos[i],
                            get_label(our_data), get_text(our_data)[:200]])

df_ours = pd.DataFrame(data=data_array_ours, columns=columns)
df_ours.to_csv('results/ours_lr_effects.csv'.format(test_pt))